In [ ]:
import numpy as np
import pandas as pd
from data_loader import load_names_from_web, births_in_year, births_per_year
import altair as alt
alt.data_transformers.disable_max_rows()

In [ ]:
dfraw = load_names_from_web(category='national', hide_pre_1937=False, use_existing_files=True)

first_known_year = dfraw['year'].min()
last_known_year = dfraw['year'].max()
births_by_year = births_per_year()

# dfraw = dfraw[dfraw['name'] == 'Sarah']

all_years = dfraw.copy()
all_years

In [ ]:
df = all_years.copy()
df = df.sort_values(by=['year', 'name'])
# df['thisyear_rank'] = df.groupby('M/F')['thisyear_count'].rank(method='first', ascending=False)
df['pct'] = df.apply(lambda row: row['count'] / births_by_year.at[row['year'], row['M/F']], axis=1)

df

In [ ]:
peaks = df.loc[df.groupby(['state', 'name', 'M/F'])['count'].idxmax()][['state', 'name', 'M/F', 'year', 'count']]
peaks = peaks.rename(columns={'year': 'peak_count_year', 'count': 'peak_count'})
df = df.merge(peaks, how='left', on=['state', 'name', 'M/F'])

df1937 = df[df['year'] >= 1937]
peaks = df1937.loc[df1937.groupby(['state', 'name', 'M/F'])['pct'].idxmax()][['state', 'name', 'M/F', 'year', 'pct']]
peaks = peaks.rename(columns={'year': 'peak_pct_year', 'pct': 'peak_pct'})
df = df.merge(peaks, how='left', on=['state', 'name', 'M/F'])

df

In [ ]:
df['pct_of_peak_count'] = df['count'] / df['peak_count']
df['pct_of_peak_pct'] = df['pct'] / df['peak_pct']
df['years_since_peak_count'] = df['year'] - df['peak_count_year']
df['years_since_peak_pct'] = df['year'] - df['peak_pct_year']
df

In [ ]:
def select_top_names(data, cutoff=10):
    '''
    Select names that have had at least one year at or above the cutoff count
    '''

    names = data[data['count'] >= cutoff][['state', 'name', 'M/F']].drop_duplicates()
    # display(names)
    data = names.merge(data, how='left', on=['state', 'name', 'M/F'])
    # display(data)

    return data

In [ ]:
backup = df.copy()

In [ ]:
data = backup.copy()
data = select_top_names(data, cutoff=1000)
data_F = data[data['M/F'] == 'F']
data_M = data[data['M/F'] == 'M']
chart_F = alt.Chart(data_F).mark_line().encode(
    x='years_since_peak_count',
    y='pct_of_peak_count',
    tooltip=['name', 'M/F', 'year', 'count', 'peak_count_year', 'peak_count'],
    color='name'
).properties(
    width=450,
    height=400
)
chart_M = alt.Chart(data_M).mark_line().encode(
    x='years_since_peak_count',
    y='pct_of_peak_count',
    tooltip=['name', 'M/F', 'year', 'count', 'peak_count_year', 'peak_count'],
    color='name'
).properties(
    width=450,
    height=400
)
chart_F | chart_M

In [ ]:
data = backup.copy()
data = select_top_names(data, cutoff=1000)
data = data[data['year'] >= 1937]
data_F = data[data['M/F'] == 'F']
data_M = data[data['M/F'] == 'M']
chart_F = alt.Chart(data_F).mark_line().encode(
    x='years_since_peak_pct',
    y='pct_of_peak_pct',
    tooltip=['name', 'M/F', 'year', 'pct', 'peak_pct_year', 'peak_pct'],
    color='name'
).properties(
    width=450,
    height=400
)
chart_M = alt.Chart(data_M).mark_line().encode(
    x='years_since_peak_pct',
    y='pct_of_peak_pct',
    tooltip=['name', 'M/F', 'year', 'pct', 'peak_pct_year', 'peak_pct'],
    color='name'
).properties(
    width=450,
    height=400
)
chart_F | chart_M